In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
import copy

In [2]:
def parse_vcf(fname):
    yield from pysam.VariantFile(fname).fetch()


def extract_vcf_stat(fname):
    ''' extract important fields from a VCF file:
        - contig name
        - position
        - reference and alternate alleles
        - strand bias, coverage, allele frequency
    '''
    return pd.DataFrame((
        {
            'contig': record.contig,
            'position': int(record.pos),
            'ref': record.alleles[0],
            'alt': alt,
            'SB': record.info["SB"],
            'DP': record.info["DP"],
            'AF': str(record.info["AF"]).split(',')[idx],
        }
        for record in parse_vcf(fname)
        for idx, alt in enumerate(record.alts)
    )
    )

In [3]:
def get_contigs(ref):
  '''@param: ref: *.fasta/fna/fa file containing the reference sequence
     @returns: a dictionary [name of contigs] -> (a string containing the entire contig sequence
  '''
  contigs = dict()
  with open(ref, "r") as f:
    lines = f.readlines()
    current_contig = ""
    contig_name = ""
    for line in lines:
      if line[0] == ">":
        if contig_name != "":
          contigs[contig_name] = current_contig
        contig_name = ((line.split()[0])[1:]).strip()
        current_contig = ""
      else: 
        current_contig += line.strip()
          # print(contig)
    contigs[contig_name] = current_contig
    f.close()
  return contigs

In [5]:
def ref_correction(vcf, ref, output, threshold=0.9, indel=False):
  ''' modify fasta to be consensus according to the vcf '''
  vcf_df = extract_vcf_stat(vcf)
  contigs = get_contigs(ref)
  modified_ref = ref[:ref.rfind("/")+1]+output+ref[ref.rfind("/")+1:]
  if not indel:
    def f(row):
      contig = row["contig"] 
      if float(row['AF']) > threshold:
        pos = row["position"]
        # print(contig, row['AF'], pos)
        contigs[contig] = contigs[contig][:pos] + row["alt"] + contigs[contig][pos+1:]
    _ = vcf_df.apply(f, axis=1)
    f = open(modified_ref,"w")
    for key in contigs.keys():
      f.write(">"+key+"\n")
      f.write(contigs[key]+"\n")
    f.close()
  else:
    print("To be implemented")

In [6]:
vcf = "./output/isolate_jumbo_read_diff_from_ref/abundant_species.filt.vcf"
ref = "./refs/fna_seq/Escherichia_marmotae.fasta"
ref_correction(vcf, ref, "modified_")

[W::vcf_parse] Contig 'NZ_CP025979.1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'NZ_CP025980.1' is not defined in the header. (Quick workaround: index the file with tabix.)


In [7]:
vcf = "./output/isolate_jumbo_read_not_very_diff_from_ref/abundant_species.filt.vcf"
ref = "./refs/fna_seq/Escherichia_marmotae.fasta"
ref_correction(vcf, ref, "modified1_")

[W::vcf_parse] Contig 'NZ_CP025979.1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'NZ_CP025980.1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'NZ_CP025981.1' is not defined in the header. (Quick workaround: index the file with tabix.)


In [ ]:
vcf = "./output/isolate_jumbo_read_diff_from_ref_indel/abundant_species.filt.vcf"
ref = "./refs/fna_seq/Escherichia_marmotae.fasta"
ref_correction(vcf, ref, "modified_indel_", indel=True)

In [ ]:
# find contig of the variant
# find original position of the variant
# if it's not the consensus base, change it